In [7]:
import pickle as pkl
import pandas as pd
import random
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [8]:
n = 1000000

In [9]:
df = pd.read_csv("formatted_data.csv", index_col = 0)
score = df["Score"].max()
score_cutoff = score * 0.8

In [10]:
with open("model.pkl", "rb") as f:
    model = pkl.load(f)

In [11]:
hp_rng = [i for i in range(400, 1001, 25)]
ex_prop_rng = [i for i in range(0, 2)]
rlg_rng = [i for i in range(0, 2)]
# empty_wgt_rng
pay_wgt_rng = [i for i in range(1700, 6001, 300)]
fuel_rng = [300]
# total_wgt_rng
wing_semi_rng = [i for i in range(25, 61, 5)]
wing_lat_rng = [2]
wing_root_rng = [j / 100.0 for j in [i for i in range(200, 651, 30)]]
wing_tip_rng = [j / 100.0 for j in [i for i in range(200, 651, 30)]]
sweep_rng = [0]
long_arm_rng = [13]
# wing_area_rng
# aspect_ratio_rng
# wing_loading_rng
# power_loading_rng

In [12]:
params = []
results = []

for i in range(0, n):
    hp = hp_rng[random.randrange(len(hp_rng))]
    ex_prop = ex_prop_rng[random.randrange(len(ex_prop_rng))]
    rlg = rlg_rng[random.randrange(len(rlg_rng))]
    # empty_wgt
    pay_wgt = pay_wgt_rng[random.randrange(len(pay_wgt_rng))]
    fuel = fuel_rng[random.randrange(len(fuel_rng))]
    # total_wgt
    wing_semi = wing_semi_rng[random.randrange(len(wing_semi_rng))]
    wing_lat = wing_lat_rng[random.randrange(len(wing_lat_rng))]
    wing_root = wing_root_rng[random.randrange(len(wing_root_rng))]
    wing_tip = wing_tip_rng[random.randrange(len(wing_tip_rng))]
    sweep = sweep_rng[random.randrange(len(sweep_rng))]
    long_arm = long_arm_rng[random.randrange(len(long_arm_rng))]
    # wing_area
    # aspect_ratio
    # wing_loading
    # power_loading

    wing_area = 2 * wing_semi * (wing_root + wing_tip) / 2
    aspect_ratio = 2 * wing_semi / ((wing_root + wing_tip) / 2)
    empty_wgt = 3835 + 20 * ex_prop + 20 * rlg + 8 * (hp - 675) + 8 * (wing_area - 257.34)
    total_wgt = empty_wgt + pay_wgt + fuel

    if total_wgt > 8000:
        continue
    
    wing_loading = total_wgt / wing_area
    power_loading = total_wgt / hp

    param = [hp, ex_prop, rlg, empty_wgt, pay_wgt, fuel, total_wgt, wing_semi, wing_lat, wing_root, wing_tip, sweep, long_arm, wing_area, aspect_ratio, wing_loading, power_loading]
    params.append(param)

    result = model.predict(np.array(param).reshape(1, -1))
    results.append(result)

    

In [13]:
results_df = pd.DataFrame(params, columns = df.columns[:-1])
results_df["Score"] = results
results_df["Score"] = results_df["Score"].astype(float)
results_df = results_df.loc[results_df["Score"] >= score_cutoff]

In [14]:
results_df.sort_values(by = "Score", ascending = False, inplace = True)
results_df.reset_index(drop = True, inplace = True)

In [15]:
results_df

,Horsepower,Extra propeller,Retractable Landing gear,Empty Weight (lb),Payload Weight (lb),Fuel Load (lb)(Enough for 40 min),Total Weight (lb),Wing 1 Semi-Length,Wing 1 Lat Arm,Wing 1 Root Chord,Wing 1 Tip Chord,Sweep,Long Arm,Wing Area,Aspect Ratio,Wing Loading,Power Loading,Score
0,400,1,0,1676.28,5900,300,7876.28,50,2,2.6,2.6,0,13,260.0,38.461538,30.293385,19.690700,95218.132759
1,400,0,0,1656.28,5600,300,7556.28,50,2,2.6,2.6,0,13,260.0,38.461538,29.062615,18.890700,95218.132759
2,425,1,0,1876.28,5600,300,7776.28,50,2,2.6,2.6,0,13,260.0,38.461538,29.908769,18.297129,95218.132759
3,400,0,1,1676.28,5900,300,7876.28,50,2,2.6,2.6,0,13,260.0,38.461538,30.293385,19.690700,95218.132759
4,400,0,1,1752.28,5900,300,7952.28,55,2,2.3,2.6,0,13,269.5,44.897959,29.507532,19.880700,95131.158507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1427,425,1,0,1588.28,5900,300,7788.28,35,2,3.2,3.2,0,13,224.0,21.875000,34.769107,18.325365,88690.979222
1428,400,0,1,1388.28,5900,300,7588.28,35,2,3.2,3.2,0,13,224.0,21.875000,33.876250,18.970700,88690.979222
1429,425,1,0,1000.28,5900,300,7200.28,35,2,2.3,2.0,0,13,150.5,32.558140,47.842392,16.941835,88688.260943
1430,425,1,0,1000.28,5900,300,7200.28,35,2,2.0,2.3,0,13,150.5,32.558140,47.842392,16.941835,88688.260943


In [16]:
results_df.to_csv("results.csv")